In [42]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/client_dataset.csv'

In [40]:
# View the column names in the data
df['first'].unique()

NameError: name 'df' is not defined

In [18]:
# Use the describe function to gather some basic statistics
df.describe()

NameError: name 'df' is not defined

In [15]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54639 entries, 0 to 54638
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   first        54639 non-null  object 
 1   last         54639 non-null  object 
 2   job          54639 non-null  object 
 3   phone        54639 non-null  object 
 4   email        54639 non-null  object 
 5   client_id    54639 non-null  int64  
 6   order_id     54639 non-null  int64  
 7   order_date   54639 non-null  object 
 8   order_week   54639 non-null  int64  
 9   order_year   54639 non-null  int64  
 10  item_id      54639 non-null  object 
 11  category     54639 non-null  object 
 12  subcategory  54639 non-null  object 
 13  unit_price   54639 non-null  float64
 14  unit_cost    54639 non-null  float64
 15  unit_weight  54639 non-null  float64
 16  qty          54639 non-null  int64  
 17  line_number  54639 non-null  int64  
dtypes: float64(3), int64(6), object(9)
memory usag

In [27]:
# What three item categories had the most entries?
df_category = df['category'].value_counts()
df_category.head(3)

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [23]:
df_category = df['category'].value_counts()
df_category

category
consumables    23538
furniture      11915
software        8400
decor           7954
electronics     2832
Name: count, dtype: int64

In [61]:
# For the category with the most entries, which subcategory had the most entries?

subcategory_counts = top_category_df['subcategory'].value_counts()
subcategory_counts

subcategory
bathroom supplies    6424
paper products       5874
kitchen supplies     5746
misc                 2891
pens                 2603
Name: count, dtype: int64

In [43]:
top_category_df = df[df['category'] == top_category]
top_category

'consumables'

In [155]:
# Which five clients had the most entries in the data?
df["client"] = df["first"] + " " + df["last"]

# Count the number of entries for each client
client_counts = df["client"].value_counts()

# Get the top 5 clients
top_5_clients = client_counts.head(5)
top_5_clients

client
Jessica Reyes      220
Angela Everett     211
Bryan Myers        209
Alexandra Young    207
Kendra Garrett     207
Name: count, dtype: int64

In [117]:
# Store the client ids of those top 5 clients in a list.
# Step 1: Count occurrences of each client ID
top_clients = df['client_id'].value_counts().head(5)

# Step 2: Extract the top 5 client IDs into a list
top_client_ids = top_clients.index.tolist()

top_client_ids


[33615, 66037, 46820, 38378, 24741]

In [145]:
# How many total units (the qty column) did the client with the most entries order order?
total_units = top_client_df['qty'].sum()
total_units
#top_client_df


64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [151]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
subtotal = df["unit_price"] * df['qty']
subtotal.head()

0    115164.00
1       523.95
2       527.28
3      1056.18
4      3902.00
dtype: float64

In [159]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.

# Calculate total weight for each order
df['total_weight'] = df['unit_weight'] * df['qty']

# Define the shipping price based on total weight
df['shipping_price'] = df['total_weight'].apply(
    lambda x: x * 7 if x > 50 else x * 10
)

# Display a preview of the updated DataFrame
df[['total_weight', 'shipping_price']].head()


,total_weight,shipping_price
0,787.50,5512.50
1,31.29,312.90
2,65.52,458.64
3,35.67,356.70
4,928.60,6500.20


In [177]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%

sales_tax_rate = 0.0925

# Calculate the subtotal (unit price * quantity)
df['subtotal'] = df['unit_price'] * df['qty']

df[['unit_price', 'qty', 'subtotal', 'sales_tax', 'shipping_price', 'total_price']].head()


,unit_price,qty,subtotal,sales_tax,shipping_price,total_price
0,1096.80,105,115164.00,10652.670000,393.750,126210.420000
1,24.95,21,523.95,48.465375,15.645,588.060375
2,13.52,39,527.28,48.773400,32.760,608.813400
3,36.42,29,1056.18,97.696650,17.835,1171.711650
4,195.10,20,3902.00,360.935000,464.300,4727.235000


In [185]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).


df['shipping_price'] = df['unit_price'] - df['unit_cost']

df['line_cost'] = (df['unit_cost'] * df['qty']) + (df['shipping_price'] * df['qty'])

df[['unit_cost', 'qty', 'shipping_price', 'line_cost']].head()


,unit_cost,qty,shipping_price,line_cost
0,762.71,105,334.09,115164.00
1,15.09,21,9.86,523.95
2,7.86,39,5.66,527.28
3,24.85,29,11.57,1056.18
4,108.17,20,86.93,3902.00


In [191]:
# Create a column for the profit of each line using line cost and line price

df['line_price'] = df['unit_price'] * df['qty']
df['line_cost'] = df['unit_cost'] * df['qty']


df['profit'] = df['line_price'] - df['line_cost']


df[['line_price', 'line_cost', 'profit']].head()


,line_price,line_cost,profit
0,115164.00,80084.55,35079.45
1,523.95,316.89,207.06
2,527.28,306.54,220.74
3,1056.18,720.65,335.53
4,3902.00,2163.40,1738.60


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [201]:
# Check your work using the totals above
# Step 1: Filter rows for each order and calculate total price
def calculate_order_total(order_id):
    order_lines = df[df['order_id'] == order_id]  
    total_price = (order_lines['unit_price'] * order_lines['qty']).sum()  
    return total_price


order_ids = [2742071, 2173913, 6128929]
order_totals = {order_id: calculate_order_total(order_id) for order_id in order_ids}


for order_id, calculated_total in order_totals.items():
  print(f"Order ID {order_id} calculated total: ${calculated_total:,.2f}")


Order ID 2742071 calculated total: $127,799.33
Order ID 2173913 calculated total: $109,440.76
Order ID 6128929 calculated total: $688,564.27


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [215]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.

# Step 1: Count the occurrences of each client_id
top_clients = df['client_id'].value_counts().head(5)

# Step 2: Store the client IDs in a list
top_client_ids = top_clients.index.tolist()

# Step 3: Display the results

top_clients
top_client_ids

[33615, 66037, 46820, 38378, 24741]

In [227]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
# Display detailed data for all clients in the DataFrame


In [18]:
# Format the data and rename the columns to names suitable for presentation.

# Define the money columns. 


# Define a function that converts a dollar amount to millions.


# Apply the currency_format_millions function to only the money columns. 


# Rename the columns to reflect the change in the money format. 




,Client ID,Units,Shipping (millions),Total Revenue (millions),Total Cost (millions),Total Profit (millions)
0,33615,64313,1.828985,8.377309,6.175314,2.201995
1,66037,43018,1.395152,10.259515,7.004483,3.255032
2,46820,75768,1.601449,9.743794,7.007192,2.736603
3,38378,73667,3.429455,12.906551,9.634721,3.271830
4,24741,239862,5.126448,82.268892,45.688900,36.579992


In [231]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
# Add a "Total Profit (millions)" column for better readability


